#**Практическая работа №1 - классификация**

*Федотьева Софья R32423*

---



#Работа с входными данными

Импорт необходимых библиотек, чтение данных из тренировочного файла

In [9]:
import os
import numpy as np
import pandas as pd

def read_data(path, filename):
    return pd.read_csv(os.path.join(path, filename))

df = read_data('/content/drive/MyDrive/Colab Notebooks', 'train.csv')
df.head()

,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,tBodyAcc-max()-X,...,fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)",subject,Activity
0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,-0.934724,...,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627,1,STANDING
1,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,-0.943068,...,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317,1,STANDING
2,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,-0.938692,...,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118,1,STANDING
3,0.279174,-0.026201,-0.123283,-0.996091,-0.983403,-0.990675,-0.997099,-0.982750,-0.989302,-0.938692,...,-0.482845,-0.036788,-0.012892,0.640011,-0.485366,-0.848649,0.181935,-0.047663,1,STANDING
4,0.276629,-0.016570,-0.115362,-0.998139,-0.980817,-0.990482,-0.998321,-0.979672,-0.990441,-0.942469,...,-0.699205,0.123320,0.122542,0.693578,-0.615971,-0.847865,0.185151,-0.043892,1,STANDING


Загрузка полного набора данных, создание необходимых переменных:
* `train_X`: признаки для обучения модели
* `train_y`: метки для обучения модели
* `test_X`: признаки для проверки модели
* `test_y`: метки для проверки модели

In [10]:
def load_dataset(label_dict):
    train_X = read_data('/content/drive/MyDrive/Colab Notebooks', 'train.csv').values[:,:-2]
    train_y = read_data('/content/drive/MyDrive/Colab Notebooks', 'train.csv')['Activity']
    train_y = train_y.map(label_dict).values
    test_X = read_data('/content/drive/MyDrive/Colab Notebooks', 'test.csv').values[:,:-2]
    test_y = read_data('/content/drive/MyDrive/Colab Notebooks', 'test.csv')
    test_y = test_y['Activity'].map(label_dict).values
    return(train_X, train_y, test_X, test_y)

label_dict = {'WALKING':0, 'WALKING_UPSTAIRS':1, 'WALKING_DOWNSTAIRS':2, 'SITTING':3, 'STANDING':4, 'LAYING':5}
train_X, train_y, test_X, test_y = load_dataset(label_dict)

#Создание и обучение моделей

Для этой практической работы я выбрала три алгоритма:
* Support Vector Machines
* Stochastic gradient descent
* k-nearest neighbors algorithm



---




подбор параметров для моделей

In [11]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from scipy.stats import reciprocal, uniform

param_distributions_svc = {"gamma": reciprocal(0.001, 0.1), "C": uniform(1, 10)}
svc = SVC()
rsc_svc = RandomizedSearchCV(estimator=svc, param_distributions=param_distributions_svc)
rsc_svc.fit(train_X, train_y)
rsc_svc.best_estimator_

SVC(C=9.317042688919267, gamma=0.012370392178733713)

In [19]:
from sklearn.neighbors import KNeighborsClassifier as knc
param_distributions_knn = {"n_neighbors": np.arange(1, 200) }
knn = knc()

rsc_knn = RandomizedSearchCV(estimator=knn, param_distributions=param_distributions_knn)
rsc_knn.fit(train_X, train_y)
rsc_knn.best_estimator_

KNeighborsClassifier(n_neighbors=49)

In [37]:
from sklearn.linear_model import SGDClassifier as sgdc
sgd = sgdc()
param_distributions_sgd = {'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3], # learning rate
    'max_iter': np.arange(1, 1000)}
rsc_sgd = RandomizedSearchCV(estimator=sgd, param_distributions=param_distributions_sgd)
rsc_sgd.fit(train_X, train_y)
rsc_sgd.best_estimator_

SGDClassifier(alpha=0.01, max_iter=379)

инициализация моделей

In [38]:
model_svc = SVC(C=9.317042688919267, gamma=0.012370392178733713)
model_knn = knc(n_neighbors=49)
model_sgd = sgdc(alpha=0.01, max_iter=379)

обучение моделей и прогнозирование



In [39]:
model_svc.fit(train_X, train_y)
yhat_svc = model_svc.predict(test_X)

model_knn.fit(train_X, train_y)
yhat_knn = model_knn.predict(test_X)

model_sgd.fit(train_X, train_y)
yhat_sgd = model_sgd.predict(test_X)

импорт библиотеки для отчета результатов классификации и создание массива target_names

In [40]:
from sklearn.metrics import classification_report
target_names = ['Walking', 'Walking Upstairs', 'Walking Downstairs', 'Sitting', 'Standing', 'Laying']

#Результаты классификации разных моделей

In [41]:
print("\nClassification report for SVM\n")
print(classification_report(test_y, yhat_svc, target_names=target_names))


Classification report for SVM

                    precision    recall  f1-score   support

           Walking       0.96      0.98      0.97       496
  Walking Upstairs       0.94      0.97      0.96       471
Walking Downstairs       0.99      0.94      0.97       420
           Sitting       0.98      0.90      0.93       491
          Standing       0.91      0.98      0.95       532
            Laying       1.00      1.00      1.00       537

          accuracy                           0.96      2947
         macro avg       0.96      0.96      0.96      2947
      weighted avg       0.96      0.96      0.96      2947



In [42]:
print("\nClassification report for KNN\n")
print(classification_report(test_y, yhat_knn, target_names=target_names))


Classification report for KNN

                    precision    recall  f1-score   support

           Walking       0.83      0.99      0.90       496
  Walking Upstairs       0.90      0.91      0.91       471
Walking Downstairs       0.98      0.75      0.85       420
           Sitting       0.92      0.76      0.83       491
          Standing       0.81      0.94      0.87       532
            Laying       1.00      0.99      1.00       537

          accuracy                           0.90      2947
         macro avg       0.91      0.89      0.89      2947
      weighted avg       0.90      0.90      0.89      2947



In [43]:
print("\nClassification report for SGD\n")
print(classification_report(test_y, yhat_sgd, target_names=target_names))


Classification report for SGD

                    precision    recall  f1-score   support

           Walking       0.90      1.00      0.95       496
  Walking Upstairs       0.96      0.91      0.94       471
Walking Downstairs       0.99      0.94      0.96       420
           Sitting       0.93      0.90      0.92       491
          Standing       0.92      0.93      0.93       532
            Laying       0.99      1.00      1.00       537

          accuracy                           0.95      2947
         macro avg       0.95      0.95      0.95      2947
      weighted avg       0.95      0.95      0.95      2947

